In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Basic Dependencies
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Read training Data
df_train = pd.read_csv("/kaggle/input/cw-clean/Train_clean.csv")
df_train.head()

In [ ]:
#Read testing data
df_test = pd.read_csv("/kaggle/input/cw-clean/Test_clean.csv")
df_test.head()

In [ ]:
#Column Types
df_train.info()

In [ ]:
#Basic Stats
df_train.describe()

In [ ]:
print(df_train.shape)
print(df_test.shape)

In [ ]:
df_train.columns

In [ ]:
def outlire_remove(data,colums):
    for col in colums:
        min_thresold, max_thresold = data[col].quantile([0.001, 0.999])
        min_thresold, max_thresold

        #retail_train = retail_train[(retail_train.InvoiceNo<22000) & (retail_train.InvoiceNo>200)]
        data = data[(data[col]<max_thresold) & (data[col]>min_thresold)]

    return data

In [ ]:
colums=['Quantity']
df_train=outlire_remove(df_train,colums)

In [ ]:
df_train.shape

In [ ]:
import seaborn as sns
sns.boxenplot(df_train['Quantity'])

In [ ]:
df_train['Quantity']=abs(df_train['Quantity'])
    

In [ ]:
sns.boxenplot(df_test['Quantity'])

In [ ]:
data = df_train

In [ ]:
#Check for Missing Values
data.isnull().sum()

In [ ]:
#Seperate Categorical and Numerical Columns
cat_cols = data.select_dtypes(include=['object','category']).columns.tolist()
print(cat_cols)

num_cols = data.select_dtypes(include=['int64','float64']).columns.tolist()
print(num_cols)

In [ ]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: data[col].nunique(), cat_cols))
d = dict(zip(cat_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [ ]:
data.head()

In [ ]:
DT -9 -StockCode Description Quantity Country Date_sin Date_cos Month_sin Month_cos Year_sin

In [ ]:
#Drop irrelavent columns
data1 = data.drop(['AM/PM_PM','Time_sin','Time_cos','CustomerID','InvoiceNo'],1, inplace= False)
data1.head()

In [ ]:
# Check the correlation of the data

In [ ]:
def plot_correlation(df, numerical_feature_columns, target, k):
    cols = df[numerical_feature_columns].corr().nlargest(k, target)[target].index
    cm = df[cols].corr()
    plt.figure(figsize=(10,6))
    return sns.heatmap(cm, annot=True, cmap = 'viridis')

def generate_x_y_numeric_target(df, target):
    numerical_feature_columns = list(df._get_numeric_data().columns)
    X = df.loc[:, df.columns != target].values
    Y = df.loc[:, df.columns == target].values
    return numerical_feature_columns, X, Y

In [ ]:
target = 'UnitPrice'
df=data1
numeric_cols, X, Y = generate_x_y_numeric_target(df, target)
plot_correlation(df, numeric_cols, target, 8)

In [ ]:
#Check for skewness in the dataset
data.skew()

In [ ]:
data.Selling_Price.hist(bins = 25)
print()
data.Selling_Price.skew()
#Highly skewed scaling needs to done before applying any regression model

In [ ]:
corr_matrix = data.corr(method='pearson')
corr_matrix

In [ ]:
#corrleation with area
corr_matrix.Selling_Price.sort_values(ascending=False)

In [ ]:
data.shape

In [ ]:
#Take targate variable into y
y = data['Selling_Price']
X = data.drop('Selling_Price',axis = 1)

from sklearn.preprocessing import PowerTransformer
y = y.values.reshape(-1,1)
# power transform the raw data
power = PowerTransformer(method='yeo-johnson', standardize=True)
y = power.fit_transform(y)

In [ ]:
# Split data into train and test format
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.pipeline import Pipeline
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', MinMaxScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', MinMaxScaler()),('LASSO', Lasso())])))
#pipelines.append(('ScaledEN', Pipeline([('Scaler', MinMaxScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledDT', Pipeline([('Scaler', MinMaxScaler()),('DT', DecisionTreeRegressor())])))
pipelines.append(('ScaledRF', Pipeline([('Scaler', MinMaxScaler()),('RF', RandomForestRegressor())])))
pipelines.append(('ScaledET', Pipeline([('Scaler', MinMaxScaler()),('ET', ExtraTreesRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', MinMaxScaler()),('GBM', GradientBoostingRegressor())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', MinMaxScaler()),('XGB', XGBRegressor())])))
#pipelines.append(('ScaledNN', Pipeline([('Scaler', MinMaxScaler()),('NN', MLPRegressor())])))
#pipelines.append(('ScaledSVR', Pipeline([('Scaler', MinMaxScaler()),('SVR', SVR(kernel='rbf'))])))
#pipelines.append(('ScaledKNN', Pipeline([('Scaler', MinMaxScaler()),('KNN', KNeighborsRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=10)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_mean_squared_error',n_jobs=-1)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
ScaledLR: -0.986794 (0.004335)
ScaledLASSO: -1.000697 (0.004855)
ScaledDT: -0.067342 (0.002947)
ScaledRF: -0.037277 (0.001732)
ScaledET: -0.023197 (0.001434)
ScaledGBM: -0.524131 (0.005523)
ScaledXGB: -0.124555 (0.002914)

In [ ]:
X_train=X_train.reset_index(drop=True, inplace=True)
y_train=y_train.reset_index(drop=True, inplace=True)
X_test=X_test.reset_index(drop=True, inplace=True)
y_test=y_test.reset_index(drop=True, inplace=True)

In [ ]:
# ScaledET is best performing 

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
ET_model = ExtraTreesRegressor()
# evaluate the model
cv = RepeatedKFold()
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise',verbose=1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
ET_model.fit(X,y)

In [ ]:
k_max = max(n_scores)
print( "At K = {}, Max Accuracy = {}".format(k_max, max(n_scores)*100))

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = ET_model.predict(X_test)
MSE = mean_squared_error(y_test , predictions)
print('ExtraTrees validation MAE = ',MSE)

In [ ]:
ExtraTrees validation MAE =  0.0001005074998879354

ExtraTrees validation MAE =  0.0017519896754034385

ExtraTrees validation MAE =  8.245282583687808e-05


ExtraTrees validation MAE =  0.00018089177342840292

In [ ]:
# Random Forest

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# define the model
RF_model = RandomForestRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=20, n_repeats=20, random_state=10)
n_scores = cross_val_score(RF_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise',verbose=1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


In [ ]:
RF_model.fit(X,y)

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = RF_model.predict(X_test)
MSE = mean_squared_error(y_test , predictions)
print('ExtraTrees validation MAE = ',MSE)

In [ ]:
# DTree

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# define the model
DT_model = DecisionTreeRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=30, n_repeats=10, random_state=None)
n_scores = cross_val_score(DT_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise',verbose=1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
DT_model.fit(X,y)

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = DT_model.predict(X_test)
MSE = mean_squared_error(y_test , predictions)
print('Desition Tree validation MAE = ',MSE)

In [ ]:
Desition Tree validation MAE =  0.0007724816644394584
Desition Tree validation MAE =  0.0008335957763079483
Desition Tree validation MAE =  0.0010537562538273323
Desition Tree validation MAE =  0.001751989675403438

In [ ]:
clf = DecisionTreeRegressor(random_state=0)
clf.fit(X_train,y_train)
pred=clf.predict(X_test)
from sklearn.metrics import accuracy_score
mean_squared_error(y_test, pred)

In [ ]:
from sklearn import tree
plt.figure(figsize=(15,10))
tree.plot_tree(DT_model,filled=True)

In [ ]:
path = DT_model.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
ccp_alphas=abs(ccp_alphas)

In [ ]:
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeRegressor(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X, y_train)
    clfs.append(clf)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      clfs[-1].tree_.node_count, ccp_alphas[-1]))

In [ ]:
import matplotlib.pyplot as plt
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker='o', label="train",
        drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker='o', label="test",
        drawstyle="steps-post")
ax.legend()
plt.show()

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_dist={'criterion': ['mse', 'friedman_mse','mae'],
                                        'max_depth': range(1, 100, 3),
                                        'max_features': range(2, 8),
                                        'min_samples_leaf': range(5, 100, 3),
                                        'min_samples_split': range(2, 100, 3),
                                        'splitter': ['best', 'random']},

In [ ]:
DT_r=DecisionTreeRegressor()

In [ ]:
random_search=RandomizedSearchCV(DT_r,n_jobs=-1,verbose=1,cv=10,n_iter=10,scoring='neg_mean_absolute_error',
                                 param_distributions=param_dist)

In [ ]:
random_search.fit (X_train,y_train)

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates=np.flatnonzero(results['rank_test_score']==i)
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean validation score: {0:3f}(std:{1:.5f})'.format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print('parameters: {0}'.format(results['params'][candidate]))
            print('')

In [ ]:
XGB=random_search.best_estimator_

In [ ]:
report(random_search.cv_results_,5)

In [ ]:
XGB.fit(X,y)

In [ ]:
XGBpredictions = XGB.predict(X_test)
MSE = mean_squared_error(y_test , XGBpredictions)
print('XGBoost validation MAE = ',MSE)

In [ ]:
# Hyper parameter tuning for DecisionTreeRegressor

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

In [ ]:
def acc_model(params):
    DTR = DecisionTreeRegressor(**params)
    return cross_val_score(DTR, X, y,n_jobs=-1).mean()

In [ ]:
param_space = {
    'max_depth': hp.choice('max_depth', range(2,100,3)),
    'min_samples_leaf': hp.choice('min_samples_leaf',range(5,100,3)),
    'min_samples_split': hp.choice('min_samples_split',range(2,100,3)),
    'max_features': hp.choice('max_features',range(2,8,1)),
    'criterion': hp.choice('criterion',['mse', 'friedman_mse','mae']),
    'splitter': hp.choice('splitter',['best', 'random']),
}

In [ ]:
best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=10, trials=trials)
print ('best:')
print (best)

In [ ]:
# XGBoost

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# define the model
XGB_model = XGBRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=20, n_repeats=20, random_state=1)
n_scores = cross_val_score(XGB_model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise',verbose=1)
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

In [ ]:
XGB_model.fit(X,y)

In [ ]:
from sklearn.metrics import mean_squared_error
predictions = XGB_model.predict(X_test)
MSE = mean_squared_error(y_test , predictions)
print('ExtraTrees validation MAE = ',MSE)

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
# Hyper parameter tuning for DecisionTreeRegressor

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score,acc

In [ ]:
def acc_model(params):
    DTR = XGBRegressor(**params)
    return cross_val_score(DTR, X, y,n_jobs=-1).mean()

In [ ]:
param_space = {
    'max_depth': hp.choice('max_depth', range(1,100)),
    
    'n_estimators': hp.choice('n_estimators', range(100,5000)),
    
    
    'min_child_weight': hp.choice('min_child_weight',range(1,10)),
    'max_delta_step': hp.choice('max_delta_step',range(1,10)),
    'scale_pos_weight':hp.choice('scale_pos_weight',range(1,50))

}

In [ ]:
best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=10, trials=trials)
print ('best:')
print (best)

In [ ]:
# Using K_Nearest neighbour regressor
# running for different K values to know which yields the max accuracy.
from sklearn.neighbors import KNeighborsRegressor
score = []
for k in range(1,20):    
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(X_train, y_train)
    score.append(clf.score(X_test, y_test))    

In [ ]:
k_max = score.index(max(score))+1
print( "At K = {}, Max Accuracy = {}".format(k_max, max(score)*100))

In [ ]:
clf = KNeighborsRegressor(n_neighbors = k_max,  weights = 'distance', p=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test ))   
y_pred = clf.predict(X_test)

In [ ]:
import joblib
# save the model to disk
filename = 'KNN_Regressor_MachineHack.sav'
joblib.dump(model, filename)

In [ ]:
#Read testing data
df_test = pd.read_csv("/kaggle/input/cw-clean/Test_clean.csv")
df_test.head()

In [ ]:
data_test = df_test.drop(['AM/PM_PM','Time_sin','Time_cos','CustomerID','InvoiceNo'],1, inplace= False)
data_test.head()

In [ ]:
X.head()

In [ ]:
# apply the whole pipeline to data
results = XGB_model.predict(df_test)

In [ ]:
results

In [ ]:
result = pd.DataFrame(data=results)
result.head()

In [ ]:

# Applying inverse power transform to get back original form of unit price
a_inverse_transformed = power.inverse_transform(result)
a_inverse_transformed

In [ ]:
result = pd.DataFrame(data=a_inverse_transformed).round(2)
result.head()

In [ ]:
result=result.rename(columns={0:'Selling_Price'})

In [ ]:
result.head()

In [ ]:
result=pd.concat(df_test['c'])

In [ ]:
result.to_csv("my_submission_file.csv",index=False)